# R2Gen学习系列 - 第1部分：数据处理与Tokenization

## 🎯 学习目标
在这个notebook中，我们将学习：
1. R2Gen项目的背景和核心思想
2. 医学报告生成的挑战
3. 文本tokenization的原理和实现
4. 词汇表构建过程
5. 数据预处理的重要性

## 📚 项目背景

### 什么是R2Gen？
R2Gen (Radiology Report Generation) 是一个基于深度学习的医学报告自动生成系统。它的核心创新是使用了**Memory-driven Transformer**架构，能够：

- 📸 **分析医学影像**：处理X光片、CT扫描等医学图像
- 🧠 **利用医学知识**：通过关系记忆模块存储和利用先验医学知识
- 📝 **生成报告**：自动生成结构化的放射学报告

### 为什么这很重要？
1. **提高效率**：减少放射科医生的工作负担
2. **标准化**：确保报告格式和术语的一致性
3. **辅助诊断**：为医生提供初步的分析结果

### 技术挑战
- 医学术语的专业性和准确性要求
- 图像特征与文本描述的对应关系
- 长序列文本生成的连贯性
- 医学知识的有效利用

## 🗺️ 学习路线图

我们将通过以下步骤逐步学习R2Gen：

```
📖 第1部分：数据处理与Tokenization (当前)
   ├── 文本预处理
   ├── Tokenization原理
   └── 词汇表构建

🖼️ 第2部分：视觉特征提取
   ├── CNN特征提取器
   ├── 图像预处理
   └── 特征可视化

🔧 第3部分：Transformer基础组件
   ├── 注意力机制
   ├── 编码器-解码器
   └── 位置编码

🧠 第4部分：关系记忆模块
   ├── 记忆机制原理
   ├── 条件层归一化
   └── 知识存储与检索

🚀 第5部分：完整模型训练
   ├── 损失函数
   ├── 训练循环
   └── 模型评估
```

## 📊 数据集介绍

R2Gen支持两个主要的医学数据集：

### 1. IU X-Ray数据集
- **图像类型**：胸部X光片
- **特点**：每个病例包含两张图像（正面和侧面）
- **报告**：结构化的放射学报告

### 2. MIMIC-CXR数据集
- **图像类型**：胸部X光片
- **特点**：大规模数据集，单张图像
- **报告**：详细的临床报告

### 数据格式示例
```json
{
  "id": "patient_001",
  "image_path": ["frontal.jpg", "lateral.jpg"],
  "report": "The lungs are clear. No acute cardiopulmonary abnormality."
}
```

In [ ]:
# 导入必要的库
import json
import re
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# 导入高分辨率配置
try:
    from matplotlib_config import setup_high_quality_plots, save_high_quality_figure, create_publication_figure, get_figure_size
    setup_high_quality_plots()
    print("✅ 已启用科研级别高分辨率绘图配置")
except ImportError:
    print("⚠️ matplotlib_config.py未找到，使用默认高分辨率配置")
    # 手动设置高分辨率
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['savefig.bbox'] = 'tight'

try:
    from wordcloud import WordCloud
    WORDCLOUD_AVAILABLE = True
except ImportError:
    WORDCLOUD_AVAILABLE = False
    print("⚠️ WordCloud库未安装，词云功能将被跳过")

# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("✅ 库导入完成！")
print("📚 准备开始学习R2Gen的数据处理模块...")

## 🔤 文本Tokenization详解

### 什么是Tokenization？
Tokenization是将原始文本转换为模型可以理解的数字序列的过程。在医学报告生成中，这个步骤尤为重要，因为：

1. **医学术语处理**：正确处理专业医学词汇
2. **标准化**：统一不同格式的报告
3. **数值化**：将文本转换为神经网络可处理的数字

### R2Gen的Tokenization流程
```
原始报告 → 文本清理 → 分词 → 词汇表映射 → 数字序列
```

In [ ]:
# 让我们从R2Gen源码中提取并简化Tokenizer类
class SimpleTokenizer:
    """简化版的R2Gen Tokenizer，用于学习目的"""
    
    def __init__(self, threshold=3):
        """
        初始化tokenizer
        
        Args:
            threshold: 词汇出现的最小频次，低于此频次的词会被标记为<unk>
        """
        self.threshold = threshold
        self.token2idx = {}  # 词汇到索引的映射
        self.idx2token = {}  # 索引到词汇的映射
        
    def clean_report_iu_xray(self, report):
        """
        清理IU X-Ray数据集的报告文本
        这个函数展示了医学文本预处理的复杂性
        """
        # 第一步：处理句子分隔符和编号
        report_cleaner = lambda t: t.replace('..', '.').replace('..', '.').replace('..', '.').replace('1. ', '') \
            .replace('. 2. ', '. ').replace('. 3. ', '. ').replace('. 4. ', '. ').replace('. 5. ', '. ') \
            .replace(' 2. ', '. ').replace(' 3. ', '. ').replace(' 4. ', '. ').replace(' 5. ', '. ') \
            .strip().lower().split('. ')
        
        # 第二步：清理每个句子中的标点符号
        sent_cleaner = lambda t: re.sub('[.,?;*!%^&_+():-\\[\\]{}]', '', 
                                       t.replace('"', '').replace('/', '').replace('\\\\', '').replace("'", '').strip().lower())
        
        # 应用清理函数
        tokens = [sent_cleaner(sent) for sent in report_cleaner(report) if sent_cleaner(sent) != '']
        report = ' . '.join(tokens) + ' .'
        
        return report
    
    def create_vocabulary(self, reports):
        """
        从报告列表中创建词汇表
        
        Args:
            reports: 报告文本列表
        """
        print("🔍 开始分析报告，构建词汇表...")
        
        # 收集所有词汇
        total_tokens = []
        for report in reports:
            cleaned_report = self.clean_report_iu_xray(report)
            tokens = cleaned_report.split()
            total_tokens.extend(tokens)
        
        # 统计词频
        counter = Counter(total_tokens)
        print(f"📊 总共发现 {len(counter)} 个不同的词汇")
        print(f"📊 总词汇数量: {len(total_tokens)}")
        
        # 过滤低频词汇
        vocab = [k for k, v in counter.items() if v >= self.threshold] + ['<unk>']
        vocab.sort()
        
        print(f"📊 过滤后词汇表大小: {len(vocab)} (阈值: {self.threshold})")
        
        # 创建映射字典
        self.token2idx = {}
        self.idx2token = {}
        
        for idx, token in enumerate(vocab):
            self.token2idx[token] = idx + 1  # 从1开始，0留给padding
            self.idx2token[idx + 1] = token
        
        return counter
    
    def tokenize(self, report):
        """
        将报告转换为token ID序列
        
        Args:
            report: 原始报告文本
            
        Returns:
            token ID列表
        """
        cleaned_report = self.clean_report_iu_xray(report)
        tokens = cleaned_report.split()
        
        ids = []
        for token in tokens:
            if token in self.token2idx:
                ids.append(self.token2idx[token])
            else:
                ids.append(self.token2idx['<unk>'])  # 未知词汇
        
        # 添加开始和结束标记
        ids = [0] + ids + [0]  # 0作为特殊标记
        return ids
    
    def decode(self, ids):
        """
        将token ID序列转换回文本
        
        Args:
            ids: token ID列表
            
        Returns:
            解码后的文本
        """
        txt = ''
        for i, idx in enumerate(ids):
            if idx > 0:  # 跳过特殊标记
                if i >= 1:
                    txt += ' '
                txt += self.idx2token[idx]
            else:
                break
        return txt

print("✅ SimpleTokenizer类定义完成！")

## 🧪 实践：使用Tokenizer处理医学报告

现在让我们用一些示例医学报告来测试我们的tokenizer！

In [ ]:
# 创建一些示例医学报告
sample_reports = [
    "The lungs are clear. No acute cardiopulmonary abnormality. Heart size is normal.",
    "Bilateral lower lobe pneumonia. Cardiomegaly is present. Pleural effusion noted.",
    "No acute findings. Lungs are clear bilaterally. Normal heart size and contour.",
    "Pneumothorax on the right side. Lung collapse observed. Immediate attention required.",
    "Chronic obstructive pulmonary disease. Hyperinflation of lungs. No acute changes.",
    "Normal chest X-ray. No abnormalities detected. Heart and lungs appear normal.",
    "Pulmonary edema present. Enlarged heart shadow. Bilateral infiltrates seen.",
    "Rib fractures on left side. No pneumothorax. Lungs are otherwise clear."
]

print("📋 示例医学报告准备完成！")
print(f"📊 总共 {len(sample_reports)} 份报告")
print("\n📝 报告示例:")
for i, report in enumerate(sample_reports[:3], 1):
    print(f"{i}. {report}")

In [ ]:
# 创建tokenizer实例并构建词汇表
tokenizer = SimpleTokenizer(threshold=2)  # 设置较低的阈值以便观察更多词汇

# 构建词汇表
word_counter = tokenizer.create_vocabulary(sample_reports)

print(f"\n📚 词汇表构建完成！")
print(f"📊 词汇表大小: {len(tokenizer.token2idx)}")
print(f"📊 最高频词汇: {word_counter.most_common(10)}")

In [ ]:
# 可视化词频分布
# 使用科研级别的图片尺寸
try:
    fig = create_publication_figure(figsize=get_figure_size('presentation'))
except NameError:
    fig = plt.figure(figsize=(12, 8), dpi=300)

# 子图1：词频分布
plt.subplot(2, 2, 1)
top_words = word_counter.most_common(15)
words, counts = zip(*top_words)
plt.barh(range(len(words)), counts)
plt.yticks(range(len(words)), words)
plt.xlabel('出现频次')
plt.title('Top 15 高频词汇')
plt.gca().invert_yaxis()

# 子图2：词汇长度分布
plt.subplot(2, 2, 2)
word_lengths = [len(word) for word in word_counter.keys()]
plt.hist(word_lengths, bins=range(1, max(word_lengths)+2), alpha=0.7, edgecolor='black')
plt.xlabel('词汇长度')
plt.ylabel('词汇数量')
plt.title('词汇长度分布')

# 子图3：词频分布直方图
plt.subplot(2, 2, 3)
freq_counts = list(word_counter.values())
plt.hist(freq_counts, bins=range(1, max(freq_counts)+2), alpha=0.7, edgecolor='black')
plt.xlabel('出现频次')
plt.ylabel('词汇数量')
plt.title('词频分布直方图')
plt.yscale('log')

# 子图4：词云（如果可用）
plt.subplot(2, 2, 4)
if WORDCLOUD_AVAILABLE:
    # 提高词云分辨率
    wordcloud = WordCloud(width=800, height=600, background_color='white', max_words=100).generate_from_frequencies(word_counter)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('医学词汇词云')
else:
    plt.text(0.5, 0.5, 'WordCloud\\n库未安装', ha='center', va='center', fontsize=16)
    plt.axis('off')
    plt.title('词云功能不可用')

plt.tight_layout()

# 保存高分辨率图片
try:
    save_high_quality_figure('vocabulary_analysis_high_res.png')
except NameError:
    plt.savefig('vocabulary_analysis_high_res.png', dpi=300, bbox_inches='tight', facecolor='white')
    print("💾 高分辨率图片已保存: vocabulary_analysis_high_res.png")

plt.show()

print("📊 词汇分析可视化完成！")
print("🎯 图片已保存为科研级别高分辨率格式 (300 DPI)")

## 🔄 Tokenization过程演示

让我们详细观察一个报告是如何被tokenize的：

In [ ]:
# 选择一个示例报告进行详细分析
example_report = "Bilateral lower lobe pneumonia. Cardiomegaly is present. Pleural effusion noted."

print("🔍 Tokenization过程演示")
print("="*50)
print(f"📝 原始报告: {example_report}")
print()

# 步骤1：文本清理
cleaned = tokenizer.clean_report_iu_xray(example_report)
print(f"🧹 清理后: {cleaned}")
print()

# 步骤2：分词
tokens = cleaned.split()
print(f"🔤 分词结果: {tokens}")
print(f"📊 词汇数量: {len(tokens)}")
print()

# 步骤3：转换为ID
token_ids = tokenizer.tokenize(example_report)
print(f"🔢 Token IDs: {token_ids}")
print()

# 步骤4：解码验证
decoded = tokenizer.decode(token_ids)
print(f"🔄 解码结果: {decoded}")
print()

# 详细的词汇映射
print("📋 详细的词汇映射:")
print("-" * 30)
for i, token in enumerate(tokens):
    token_id = tokenizer.token2idx.get(token, tokenizer.token2idx['<unk>'])
    print(f"{i+1:2d}. '{token}' → {token_id}")

## 🎓 本节总结

### 🎯 我们学到了什么？

1. **R2Gen项目背景**
   - 医学报告自动生成的重要性
   - Memory-driven Transformer的核心思想
   - 技术挑战和解决方案

2. **文本预处理的重要性**
   - 医学文本的特殊性
   - 标准化处理的必要性
   - 清理规则的设计原则

3. **Tokenization核心概念**
   - 词汇表构建策略
   - 频次阈值的作用
   - 未知词汇的处理

4. **实践技能**
   - 实现简化版tokenizer
   - 词频分析和可视化
   - 性能评估方法

### 🔑 关键要点

- **数据质量决定模型性能**：好的预处理是成功的一半
- **领域特异性**：医学文本需要专门的处理策略
- **平衡性考虑**：词汇表大小与覆盖率的权衡
- **可解释性**：每个处理步骤都应该是可理解和可验证的

### 🚀 下一步学习计划

在下一个notebook中，我们将学习：

📸 **视觉特征提取**
- CNN特征提取器的工作原理
- 医学图像的预处理
- 特征可视化和分析
- ResNet架构详解

### 💡 思考题

1. 为什么医学文本需要特殊的清理规则？
2. 如何选择合适的词频阈值？
3. OOV问题对模型性能有什么影响？
4. 如何改进tokenizer以更好地处理医学术语？

### 📚 扩展阅读

- [Subword tokenization方法](https://huggingface.co/docs/tokenizers/)
- [医学NLP的特殊挑战](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6568148/)
- [R2Gen原论文](https://arxiv.org/pdf/2010.16056.pdf)

---

🎉 **恭喜您完成了R2Gen学习系列的第一部分！**

您现在已经掌握了文本预处理和tokenization的核心概念，为后续学习打下了坚实的基础。继续保持学习的热情，我们在下一个notebook中见！